In [1]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc

In [2]:
# Define variables
iso3 = 'KEN'

# Define global datasets
ntl_aws_base = 's3://wbgdecinternal-ntl/NTL/VIIRS/Annual/VIIRS_ANNUAL_EOG/'
ntl_files = [
    'VNL_v2_npp_201204-201303_global_vcmcfg_c202101211500.average.tif',
    'VNL_v2_npp_2013_global_vcmcfg_c202101211500.average.tif',
    'VNL_v2_npp_2014_global_vcmslcfg_c202101211500.average.tif',
    'VNL_v2_npp_2015_global_vcmslcfg_c202101211500.average.tif',
    'VNL_v2_npp_2016_global_vcmslcfg_c202101211500.average.tif',
    'VNL_v2_npp_2017_global_vcmslcfg_c202101211500.average.tif',
    'VNL_v2_npp_2018_global_vcmslcfg_c202101211500.average.tif',
    'VNL_v2_npp_2019_global_vcmslcfg_c202101211500.average.tif',
    'VNL_v2_npp_2020_global_vcmslcfg_c202101211500.average.tif'
]
admin0_layer = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
smod_global = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_SMOD/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0.tif"
urban_centres = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_STAT_UCDB2015MT_GLOBE_R2019A/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg"
ghsl_global = "/home/public/Data/GLOBAL/GHSL/ghsl.vrt"

# Define input data
in_folder = "/home/wb411133/data/Projects/KEN_Urban_Change/Data"
in_aoi = os.path.join(in_folder, "Nairobi_AOI.shp")

# Define data to extract
smod_local = os.path.join(in_folder, 'ghs_smod.tif')
ntl_2013   = os.path.join(in_folder, 'ntl_2014.tif')
ntl_2020   = os.path.join(in_folder, 'ntl_2020.tif')
ntl_diff   = os.path.join(in_folder, 'ntl_diff_2014_2020.tif')
local_cities = os.path.join(in_folder, 'local_cities.geojson')
local_ghsl = os.path.join(in_folder, 'ghsl.tif')

In [3]:
# Read in base data
inAOI = gpd.read_file(in_aoi)
inA = gpd.read_file(admin0_layer)
inA = inA.loc[inA['ISO3'] == iso3]

In [14]:
importlib.reload(rMisc)
# Clip data
if not os.path.exists(smod_local):
    smodR = rasterio.open(smod_global)
    rMisc.clipRaster(smodR, inA, smod_local)
    
if not os.path.exists(ntl_2013):
    ntl0 = rasterio.open(os.path.join(ntl_aws_base, ntl_files[2]))
    rMisc.clipRaster(ntl0, inA, ntl_2013)
    
    ntl1 = rasterio.open(os.path.join(ntl_aws_base, ntl_files[-1]))
    rMisc.clipRaster(ntl1, inA, ntl_2020)
    
if not os.path.exists(ntl_diff):
    ntl0 = rasterio.open(ntl_2013)
    ntl1 = rasterio.open(ntl_2020)
    
    ntl0_data = ntl0.read()
    ntl1_data = ntl1.read()
    
    ntl_d = ntl1_data - ntl0_data

    with rasterio.open(ntl_diff, 'w', **ntl0.meta) as outR:
        outR.write(ntl_d)
        
if not os.path.exists(local_cities):
    inU = gpd.read_file(urban_centres)
    selU = inU.loc[inU['CTR_MN_ISO'] == iso3]
    selU.to_file(local_cities)
    
if not os.path.exists(local_ghsl):
    ghslR = rasterio.open(ghsl_global)
    selU = gpd.read_file(local_cities)
    selU = selU.loc[selU['UC_NM_MN'].isin(["Nairobi",'Ruaka','Ndenderu','Kiambu','Ruiru'])]
    rMisc.clipRaster(ghslR, selU, local_ghsl, crop=False)